In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('C:/Users/maste/Documents/대학교/대학교 3학년 2학기/기계학습/Final Project/movies.csv')

In [3]:
movies = movies.drop(movies.columns[0],axis=1)

## Similarity Between Movies

### Cosine Similarity

In [4]:
from scipy import spatial
import ast  # Import the ast module for literal_eval

def convert_to_binary_list(binary_str):
    # Convert a string representation of a list to an actual list
    try:
        binary_list = ast.literal_eval(binary_str)
        return binary_list
    except (SyntaxError, ValueError):
        # Handle the case where the string cannot be evaluated as a list
        return []

def Similarity(movieId1, movieId2):
    a = movies.iloc[movieId1]
    b = movies.iloc[movieId2]
    
    genresA = convert_to_binary_list(a['binary_genres'])
    genresB = convert_to_binary_list(b['binary_genres'])
    
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    castA = convert_to_binary_list(a['binary_cast'])
    castB = convert_to_binary_list(b['binary_cast'])
    castDistance = spatial.distance.cosine(castA, castB)
    
    directA = convert_to_binary_list(a['binary_director'])
    directB = convert_to_binary_list(b['binary_director'])
    directDistance = spatial.distance.cosine(directA, directB)
    
    wordsA = convert_to_binary_list(a['binary_words'])
    wordsB = convert_to_binary_list(b['binary_words'])
    wordsDistance = spatial.distance.cosine(wordsA, wordsB)
    
    return genreDistance + directDistance + castDistance + wordsDistance

### Recommend Movie

In [5]:
import operator

def recommend_movie(name):
    new_movie = movies[movies['original_title'].str.contains(name)].iloc[0].to_frame().T
    
    print('Selected Movie: ', new_movie.original_title.values[0])
    
    def getNeighbors(baseMovie, K):
        distances = []

        for index, movie in movies.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0].astype(int):
                dist = Similarity(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))

        distances.sort(key=operator.itemgetter(1))
        neighbors = []

        for x in range(min(K, len(distances))):
            neighbors.append(distances[x])

        return neighbors
    
    K = 10
    neighbors = getNeighbors(new_movie, K)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        print(
        str(movies.iloc[neighbor[0]][1]) +
        " | Genres: " +
        str(movies.iloc[neighbor[0]][2]).strip('[]').replace(' ', '') +
        " | Rating: " +
        str(movies.iloc[neighbor[0]][4]))

In [6]:
recommend_movie('The Dark Knight')

Selected Movie:  The Dark Knight Rises


C:\Users\maste\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



Recommended Movies: 

The Dark Knight | Genres: 'Action','Crime','Drama','Thriller' | Rating: 8.2
Batman Begins | Genres: 'Action','Crime','Drama' | Rating: 7.5
The Prestige | Genres: 'Drama','Mystery','Thriller' | Rating: 8.0
Insomnia | Genres: 'Crime','Mystery','Thriller' | Rating: 6.8
Inception | Genres: 'Action','Adventure','Mystery','ScienceFiction','Thriller' | Rating: 8.1
Interstellar | Genres: 'Adventure','Drama','ScienceFiction' | Rating: 8.1
Memento | Genres: 'Mystery','Thriller' | Rating: 8.1
Takers | Genres: 'Action','Crime','Drama','Thriller' | Rating: 6.0
Mercury Rising | Genres: 'Action','Crime','Drama','Thriller' | Rating: 6.0
Harry Brown | Genres: 'Action','Crime','Drama','Thriller' | Rating: 6.7
